In [67]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import pycaw

import cv2
import numpy as np
import time
import mediapipe 
from cvzone.FPS import FPS
import os
import pyautogui as control
control.FAILSAFE =False
import math
import csv
import time

############# our Variables ############
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
startDistance = None
cx,cy =200,200


############################### 
detector = HandDetector(staticMode=False,maxHands=2,modelComplexity=1,detectionCon=0.5, minTrackCon=0.5)   
############################################
while True :
    _,frame = cap.read()
    frame = cv2.flip(frame,1)
    hands , _ = detector.findHands(frame,draw=True,flipType=False)
    image = cv2.imread('geka.png')

    #1-check number of hands is 2
    if len(hands )==2:
        #2- check if finger 8 and finger 0 is up for both hands 
        hand1 = hands[0]
        hand2 = hands[1]
        hand1list = hand1['lmList']
        hand2list = hand2['lmList']
        if detector.fingersUp(hand1)==[0,1,0,0,0] and detector.fingersUp(hand2)==[0,1,0,0,0]:
            #3- claculate ditance between tip of finger of 8 in both hands
            #4 - check is start distance is none
            #5 if none  get value of initial valu of ditance 
            if startDistance is None :
                length ,info ,_ = detector.findDistance(hand1list[8][0:2],hand2list[8][0:2],frame)
                #put image in center  between to hands insteadof finger 8
                #length ,info ,_ = detector.findDistance(hand1['center'],hand2['center'],frame)
                startDistance = length
            #caluclate the iterated value od ditance 
            length ,info ,_ = detector.findDistance(hand1list[8][0:2],hand2list[8][0:2],frame)
            #put image in center  between to hands insteadof finger 8
            #length ,info ,_ = detector.findDistance(hand1['center'],hand2['center'],frame)

            zoomValue = int((length - startDistance)//2)
            #6- calculate cx ,cy in order to put image inside it
            cx,cy = info[4],info[5]

    else:
        startDistance = None
        
    #7 - start get image shape (h,w)
    try:    
        h1 ,w1 ,_= image.shape
        #8- get new size 
        newH =int((h1+zoomValue)//2)*2
        newW =int ((w1+zoomValue)//2)*2
        image =cv2.resize(image,(newW,newH))
        frame[cy-newH//2:cy+ newH//2, cx-newW//2:cx+ newW//2] = image
    except :
        pass

    imgPNG = cv2.imread("C:/Users/Geka/Desktop/update.png",cv2.IMREAD_UNCHANGED)
    imgOverlay = cvzone.overlayPNG(frame, imgPNG, pos=[3, 430])
    cv2.imshow('Viruial Zoom In & Out Project',frame)
    # Display the camera feed
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 
